In [1]:
import torch
import pickle

from learner import Learner
from meta_learner import Meta_Learner
from mydataset.stanforddogs import StanFordDogs
from learning import *

In [2]:
class MyArgs:
    def __init__(
        self,
        epoch=10,
        n_way=5,
        k_spt=1,
        k_qry=15,
        imgsz=84,
        imgc=3,
        task_num=4,
        meta_lr=1e-3,
        update_lr=0.01,
        update_step=5,
        update_step_test=10,
    ):
        self.epoch = epoch
        self.n_way = n_way
        self.k_spt = k_spt
        self.k_qry = k_qry
        self.imgsz = imgsz
        self.imgc = imgc
        self.task_num = task_num
        self.meta_lr = meta_lr
        self.update_lr = update_lr
        self.update_step = update_step
        self.update_step_test = update_step_test

## 5-way 1-shot
+ query : 5-way 15-shot

In [3]:
args = MyArgs(epoch=10,
              n_way=5,
              k_spt=1,
              k_qry=15,
              imgsz=84,
              imgc=3,
              task_num=8,
              meta_lr=1e-3,
              update_lr=0.01,
              update_step=5,
              update_step_test=10)

config = [
    ("conv2d", [32, 3, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [32, 32, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [32, 32, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [32, 32, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 1, 0]),
    ("flatten", []),
    ("linear", [args.n_way, 32 * 5 * 5]),
]


device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
maml = Meta_Learner(args, config, task='classification').to(device)

print(maml)

Meta_Learner(
  (net): Learner(
    conv2d:(ch_in:3, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:1, padding:0)
    flatten:()
    linear:(in:800, out:5)
    
    (vars): ParameterList(
        (0): Parameter containing: [torch.cuda.FloatTensor of size 32x3x3x3 (GPU 0)]
        (1): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (2): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (3): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (4): Parameter containing: [torch

In [4]:
# batchsz here means total episode number
train_db = StanFordDogs(
    "stanforddogs/",
    mode="train",
    n_way=args.n_way,
    k_shot_spt=args.k_spt,
    k_shot_qry=args.k_qry,
    batch_size=7500,
    resize=args.imgsz,
)
valid_db = StanFordDogs(
    "stanforddogs/",
    mode="valid",
    n_way=args.n_way,
    k_shot_spt=args.k_spt,
    k_shot_qry=args.k_qry,
    batch_size=75,
    resize=args.imgsz,
)
test_db = StanFordDogs(
    "stanforddogs/",
    mode="test",
    n_way=args.n_way,
    k_shot_spt=args.k_spt,
    k_shot_qry=args.k_qry,
    batch_size=150,
    resize=args.imgsz,
)

StanFord Dogs Dataset(train) :
	Batch_size : 7500
	Support sets : 5-way 1-shot
	Query sets : 5-way 15-shot
	Resizing Image : 84x84
StanFord Dogs Dataset(valid) :
	Batch_size : 75
	Support sets : 5-way 1-shot
	Query sets : 5-way 15-shot
	Resizing Image : 84x84
StanFord Dogs Dataset(test) :
	Batch_size : 150
	Support sets : 5-way 1-shot
	Query sets : 5-way 15-shot
	Resizing Image : 84x84


In [5]:
history = train(maml, device, args.epoch, args.task_num, train_db, valid_db, 32, 64)

938it [47:24,  3.03s/it, epoch=1/10, loss=1.536968, acc=0.286]
Mean Train Loss : 1.633519
Mean Train acc  : 0.2835
Mean Valid Loss : 1.617188
Mean Valid acc  : 0.2952

938it [49:09,  3.14s/it, epoch=2/10, loss=1.493005, acc=0.324]
Mean Train Loss : 1.512830
Mean Train acc  : 0.3239
Mean Valid Loss : 1.536133
Mean Valid acc  : 0.3206

938it [49:32,  3.17s/it, epoch=3/10, loss=1.583822, acc=0.279]
Mean Train Loss : 1.494298
Mean Train acc  : 0.3360
Mean Valid Loss : 1.520508
Mean Valid acc  : 0.3328

938it [46:35,  2.98s/it, epoch=4/10, loss=1.468338, acc=0.338]
Mean Train Loss : 1.467469
Mean Train acc  : 0.3524
Mean Valid Loss : 1.508789
Mean Valid acc  : 0.3450

938it [46:28,  2.97s/it, epoch=5/10, loss=1.469725, acc=0.371]
Mean Train Loss : 1.407126
Mean Train acc  : 0.3963
Mean Valid Loss : 1.500977
Mean Valid acc  : 0.3550

938it [49:26,  3.16s/it, epoch=6/10, loss=1.378874, acc=0.399]
Mean Train Loss : 1.360504
Mean Train acc  : 0.4149
Mean Valid Loss : 1.499023
Mean Valid acc  : 

In [6]:
test_result = evaluate(maml, device, test_db, mode='test')

print('loss mean : {}'.format([round(val, 5) for val in test_result[0]]))
print('acc mean  : {}'.format([round(val, 3) for val in test_result[1]]))
print('Total loss : {:.6f}'.format(test_result[2]))
print('Total acc  : {:.4f}'.format(test_result[3]))

150it [01:23,  1.79it/s, loss=1.647681, acc=0.265]
loss mean : [1.7041, 1.52051, 1.52637, 1.53223, 1.53711, 1.54102, 1.5459, 1.54883, 1.55273, 1.55566, 1.55859]
acc mean  : [0.194, 0.373, 0.374, 0.373, 0.374, 0.375, 0.374, 0.374, 0.374, 0.374, 0.374]
Total loss : 1.556641
Total acc  : 0.3574


In [7]:
with open('history/stanforddogs_5-way_1-shot_train_history.pickle', 'wb') as f:
    pickle.dump(history, f, pickle.HIGHEST_PROTOCOL)
    
with open('history/stanforddogs_5-way_1-shot_test_result.pickle', 'wb') as f:
    pickle.dump(test_result, f, pickle.HIGHEST_PROTOCOL)

## 5-way 5-shot
+ query : 5-way 15-shot

In [3]:
args = MyArgs(epoch=10,
              n_way=5,
              k_spt=5,
              k_qry=15,
              imgsz=84,
              imgc=3,
              task_num=8,
              meta_lr=1e-3,
              update_lr=0.01,
              update_step=5,
              update_step_test=10)

config = [
    ("conv2d", [32, 3, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [32, 32, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [32, 32, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 2, 0]),
    ("conv2d", [32, 32, 3, 3, 1, 0]),
    ("relu", [True]),
    ("bn", [32]),
    ("max_pool2d", [2, 1, 0]),
    ("flatten", []),
    ("linear", [args.n_way, 32 * 5 * 5]),
]


device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
maml = Meta_Learner(args, config, task='classification').to(device)

print(maml)

Meta_Learner(
  (net): Learner(
    conv2d:(ch_in:3, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:1, padding:0)
    flatten:()
    linear:(in:800, out:5)
    
    (vars): ParameterList(
        (0): Parameter containing: [torch.cuda.FloatTensor of size 32x3x3x3 (GPU 0)]
        (1): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (2): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (3): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (4): Parameter containing: [torch

In [4]:
# batchsz here means total episode number
train_db = StanFordDogs(
    "stanforddogs/",
    mode="train",
    n_way=args.n_way,
    k_shot_spt=args.k_spt,
    k_shot_qry=args.k_qry,
    batch_size=7500,
    resize=args.imgsz,
)
valid_db = StanFordDogs(
    "stanforddogs/",
    mode="valid",
    n_way=args.n_way,
    k_shot_spt=args.k_spt,
    k_shot_qry=args.k_qry,
    batch_size=75,
    resize=args.imgsz,
)
test_db = StanFordDogs(
    "stanforddogs/",
    mode="test",
    n_way=args.n_way,
    k_shot_spt=args.k_spt,
    k_shot_qry=args.k_qry,
    batch_size=150,
    resize=args.imgsz,
)

StanFord Dogs Dataset(train) :
	Batch_size : 7500
	Support sets : 5-way 5-shot
	Query sets : 5-way 15-shot
	Resizing Image : 84x84
StanFord Dogs Dataset(valid) :
	Batch_size : 75
	Support sets : 5-way 5-shot
	Query sets : 5-way 15-shot
	Resizing Image : 84x84
StanFord Dogs Dataset(test) :
	Batch_size : 150
	Support sets : 5-way 5-shot
	Query sets : 5-way 15-shot
	Resizing Image : 84x84


In [5]:
history = train(maml, device, args.epoch, args.task_num, train_db, valid_db, 32, 64)

938it [59:31,  3.81s/it, epoch=1/10, loss=1.461149, acc=0.356]
Mean Train Loss : 1.496084
Mean Train acc  : 0.3627
Mean Valid Loss : 1.428711
Mean Valid acc  : 0.3909

938it [1:00:30,  3.87s/it, epoch=2/10, loss=1.190817, acc=0.504]
Mean Train Loss : 1.335887
Mean Train acc  : 0.4350
Mean Valid Loss : 1.325195
Mean Valid acc  : 0.4463

938it [1:00:14,  3.85s/it, epoch=3/10, loss=1.121486, acc=0.543]
Mean Train Loss : 1.167969
Mean Train acc  : 0.5228
Mean Valid Loss : 1.291992
Mean Valid acc  : 0.4756

938it [59:34,  3.81s/it, epoch=4/10, loss=0.988235, acc=0.616]
Mean Train Loss : 1.034455
Mean Train acc  : 0.5772
Mean Valid Loss : 1.308594
Mean Valid acc  : 0.4719

938it [59:46,  3.82s/it, epoch=5/10, loss=0.869003, acc=0.659]
Mean Train Loss : 0.955954
Mean Train acc  : 0.6129
Mean Valid Loss : 1.377930
Mean Valid acc  : 0.4597

938it [59:23,  3.80s/it, epoch=6/10, loss=0.886621, acc=0.653]
Mean Train Loss : 0.975983
Mean Train acc  : 0.6272
Mean Valid Loss : 1.466797
Mean Valid acc

In [6]:
test_result = evaluate(maml, device, test_db, mode='test')

print('loss mean : {}'.format([round(val, 5) for val in test_result[0]]))
print('acc mean  : {}'.format([round(val, 3) for val in test_result[1]]))
print('Total loss : {:.6f}'.format(test_result[2]))
print('Total acc  : {:.4f}'.format(test_result[3]))

150it [01:08,  2.19it/s, loss=1.484426, acc=0.439]
loss mean : [2.04297, 1.39844, 1.37891, 1.37988, 1.38574, 1.3916, 1.39648, 1.40137, 1.40527, 1.41016, 1.41406]
acc mean  : [0.197, 0.474, 0.479, 0.481, 0.483, 0.486, 0.485, 0.487, 0.487, 0.488, 0.488]
Total loss : 1.455078
Total acc  : 0.4578


In [7]:
with open('history/stanforddogs_5-way_5-shot_train_history.pickle', 'wb') as f:
    pickle.dump(history, f, pickle.HIGHEST_PROTOCOL)
    
with open('history/stanforddogs_5-way_5-shot_test_result.pickle', 'wb') as f:
    pickle.dump(test_result, f, pickle.HIGHEST_PROTOCOL)